In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing_extensions import TypedDict, Annotated
from langgraph.graph import START, StateGraph, END
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
class SharedState(TypedDict):
    github_handle: str
    github_profile_content: str
    github_profile_summary: str
    model: ChatOpenAI

In [6]:
def build_model(shared_state: SharedState) -> SharedState:
    model = ChatOpenAI(model= "gpt-5")
    shared_state['model'] = model

    return shared_state

In [7]:
def pull_github_profile_content(shared_state: SharedState) -> SharedState:
    '''Pulls the GitHub profile content for the given GitHub handle.'''
    print('Pulling GitHub profile content...')
    github_profile_url = 'https://www.github.com/' + shared_state['github_handle']
    documents = WebBaseLoader(github_profile_url).load()
    page_content = ''

    for document in documents:
        page_content += document.page_content
    
    shared_state['github_profile_content'] = page_content.strip()
    return shared_state

In [8]:
def summarize_github_profile(shared_state: SharedState) -> SharedState:
    '''Summarizes the GitHub profile content.'''
    print('Summarizing GitHub profile content...')
    model = shared_state['model']
    github_profile_content = shared_state['github_profile_content']

    prompt = f'''Summarize the following GitHub profile content in a concise manner
    and highlight name, organization, followers, location, contact information key skills, 
    projects, and contributions.

    Github Profile Content: {github_profile_content}
    '''
    response = model.invoke(prompt)
    shared_state['github_profile_summary'] = response.content.strip()

    return shared_state


In [9]:
def build_serial_graph():
  # Building a Graph
  # State of the Graph that will be shared among nodes.
  workflow = StateGraph(SharedState)

  # Add nodes.
  workflow.add_node("build_model", build_model)
  workflow.add_node("pull_github_profile_content", pull_github_profile_content)
  workflow.add_node("summarize_github_profile", summarize_github_profile)

  # Define the edges of the graph.
  workflow.add_edge(START, "build_model")
  workflow.add_edge("build_model", "pull_github_profile_content")
  workflow.add_edge("pull_github_profile_content", "summarize_github_profile")
  workflow.add_edge("summarize_github_profile", END)

  graph = workflow.compile()
  
  return graph

In [21]:
load_dotenv()

compiled_graph = build_serial_graph()
response = compiled_graph.invoke({
    'github_handle': 'SauravP97'
})

print(response['github_profile_summary'])

Pulling GitHub profile content...
Summarizing GitHub profile content...
Here’s a concise summary of the GitHub profile:

- Name: Saurav Prateek (username: SauravP97)
- Organization/Role: Web Solutions Engineer II at Google (gTech); ex-Software Engineer at GeeksForGeeks
- Followers: 163 (following 10)
- Location: Gurugram, India
- Contact:
  - LinkedIn: https://www.linkedin.com/in/saurav-prateek-7b2096140/
  - Also runs the “Systems That Scale” newsletter (30,000+ subscribers) and a YouTube channel (Distributed Systems 101; RAG & LangChain multi-agent workflows)
- Key Skills:
  - Java, TypeScript
  - AI/ML and LLMs (RAG, LangChain, agentic workflows)
  - Distributed systems, low-level design
  - Data structures and algorithms
  - Web solutions engineering
- Notable Projects (pinned):
  - Saurav-Low-Level-Design-Template (Java) — 121★, 35 forks — Low-level design template
  - Saurav-s-DSA-Templates (Java) — 51★, 31 forks — DSA templates
  - LangPost (TypeScript) — 39★, 4 forks — Agent th

In [14]:
print(build_serial_graph().get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	build_model(build_model)
	pull_github_profile_content(pull_github_profile_content)
	summarize_github_profile(summarize_github_profile)
	__end__([<p>__end__</p>]):::last
	__start__ --> build_model;
	build_model --> pull_github_profile_content;
	pull_github_profile_content --> summarize_github_profile;
	summarize_github_profile --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

